In [1]:
import os
import math
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from sklearn import metrics
from sklearn.model_selection import KFold, train_test_split
from scipy.stats import pearsonr
#from dataset import Dataset, collate_fn
SEED = 2333

C:\Users\moham\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import os

import pickle


i = 0

feature_dir = 'Feature_vector//Feature_vector//'
label_dir = 'Label//Label//'



folders = os.listdir(feature_dir)
Atomic_wild = []
labels = []
for i in range(len(folders)):
    Atomic_wild.append(pickle.load(open(feature_dir + folders[i] +  '//vectors.pkl'  , "rb")))
    labels.append(pickle.load(open(label_dir + folders[i] +  '//label.pkl'  , "rb"))[0,-1])
    

#vectors = pickle.load(open(feature_dir + folders[i] +  '//vectors.pkl'  , "rb"))

#labels = pickle.load(open(label_dir + folders[i] +  '//label.pkl'  , "rb"))[0,-1]

In [10]:
feature_dir = 'Feature_vector//Feature_vector'
folders = os.listdir(feature_dir)

In [16]:
len(Atomic_wild)

719

In [17]:
sequence_names= folders

In [18]:
sequence_names

['0096f71c-7777-417c-8d95-6fa1699879ec',
 '00be20a3-6ec8-44dd-afac-9085c6a03da9',
 '00ca6d3f-b495-4711-8ac3-d24175f5ee9f',
 '00f836a8-9b94-4413-95eb-fd127ee723b2',
 '01000de7-d0e8-4bb4-9918-4cbcd5bbfa3d',
 '0108fa63-6200-4896-8da9-0dbbd20998de',
 '011db747-84f4-4e3e-b446-1b667bef0ea9',
 '018aaaa8-78db-4341-8811-7356a871a2fe',
 '0222e12c-539f-4640-8007-fbd625fc0ea8',
 '024c0ec6-2e01-4640-a71c-6116e3a275ff',
 '029972c6-45c3-4a08-baf2-82a3b377fb71',
 '02b9de3a-c7ee-46ff-b0b7-e0561e6b3101',
 '0309db59-8d00-4c20-b816-015a3576f059',
 '039256d5-14c1-4057-b535-01416d2ba5ac',
 '0492d6eb-5192-45ca-949f-958e9f22c29d',
 '05042790-054b-43ea-a8bc-a8dd524c4627',
 '05792b42-0953-48ea-a4b1-3e9a1691677b',
 '0592845a-01d7-4d42-8149-72af28b83c53',
 '068e7a7f-7db9-4769-8052-33c03e1f7f32',
 '06a5cbd5-f9f4-466f-9f5a-a3fdea74f270',
 '07cee6c6-be65-47dd-b1d3-7a3311dec0a6',
 '07d54db2-b54b-4f87-a281-68f0de5328bb',
 '0834dfbb-1b2b-4125-a95b-d43a829c6fc7',
 '084893ca-b836-4824-a9b1-590980679155',
 '087647cd-4f1c-

In [2]:
import gzip
import numpy as np

f = gzip.GzipFile('my_vector.npy.gz', "r" )
Atomic_wild = list(np.load(f,allow_pickle=True))
labels = np.load('final_label.npy')

In [3]:
nums1 = []
for i in range (1,720) :
      nums1 += [ i ]
nums1.sort()
#print (nums1)

#c = str(nums1)

In [4]:
c = []
for i in range(len(nums1)):
    c.append(str(nums1[i]))

In [19]:
Model_Path = './Model/'
Result_Path = './Result/'

In [20]:
WG = dict(zip(sequence_names, Atomic_wild))
zipped = list(zip(sequence_names, labels))
ds = pd.DataFrame(zipped, columns=['names', 'stability'])

In [21]:
def NodeM(name):
    return WG[name]

In [22]:
names = ds['names'].values.tolist()

In [23]:
from torch.utils.data.sampler import Sampler
  
class Dataset(Dataset):

    def __init__(self, dataframe):
        self.names = dataframe['names'].values.tolist()
        self.labels = dataframe['stability'].values.tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        
        sequence_name = self.names[index]
        label = self.labels[index]
        
        sequence_feature = NodeM(sequence_name)


        sample = {'sequence_feature': sequence_feature,\
                  'label': label, \
                  'sequence_name': sequence_name, \
                  }
        return sample
def collate_fn(batch):
    sequence_feature = []
    sequence_names = [] 
    labels=[]   
    for i in range(len(batch)):
        sequence_feature.append(batch[i]['sequence_feature'])
        sequence_feature=np.asarray(sequence_feature)
        sequence_names.append(batch[i]['sequence_name'])
        labels.append(batch[i]['label'])
        labels= np.asarray(labels)

    sequence_feature = torch.from_numpy(sequence_feature).float()
    labels= torch.from_numpy(labels)

    return sequence_feature, labels, sequence_names
def collate_MIL(batch):
	img = torch.cat([item[0] for item in batch], dim = 0)
	label = torch.LongTensor([item[1] for item in batch])
	return [img, label]


def initialize_weights(module):
	for m in module.modules():
		if isinstance(m, nn.Linear):
			nn.init.xavier_normal_(m.weight)
			m.bias.data.zero_()
		
		elif isinstance(m, nn.BatchNorm1d):
			nn.init.constant_(m.weight, 1)
			nn.init.constant_(m.bias, 0)

In [69]:
# Model parameters
NUMBER_EPOCHS = 75
LEARNING_RATE = 1E-5
WEIGHT_DECAY = 1E-7
BATCH_SIZE = 1

DENSE_DIM = 16
ATTENTION_HEADS = 4


In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pdb
#from utils.utils import initialize_weights
import numpy as np


In [26]:
class Attn_Net_Gated(nn.Module):

    def __init__(self, L = 2560, D = 256, dropout = False, n_classes = 1):
        super(Attn_Net_Gated, self).__init__()
        self.attention_a = [
            nn.Linear(L, D),
            nn.Tanh()]
        
        self.attention_b = [nn.Linear(L, D),
                            nn.Sigmoid()]
        if dropout:
            self.attention_a.append(nn.Dropout(0.25))
            self.attention_b.append(nn.Dropout(0.25))

        self.attention_a = nn.Sequential(*self.attention_a)
        self.attention_b = nn.Sequential(*self.attention_b)
        
        self.attention_c = nn.Linear(D, n_classes)

    def forward(self, x):
        a = self.attention_a(x)
        b = self.attention_b(x)
        A = a.mul(b)
        A = self.attention_c(A)  # N x n_classes
        return A, x

In [27]:
class Attn_Net(nn.Module):

    def __init__(self, L = 1024, D = 256, dropout = False, n_classes = 1):
        super(Attn_Net, self).__init__()
        self.module = [
            nn.Linear(L, D),
            nn.Tanh()]

        if dropout:
            self.module.append(nn.Dropout(0.25))

        self.module.append(nn.Linear(D, n_classes))
        
        self.module = nn.Sequential(*self.module)
    
    def forward(self, x):
        return self.module(x), x # N x n_classes

In [28]:
import torch.nn.functional as F
class Model(nn.Module):
   # def __init__(self):
        #super(Model, self).__init__()
    def __init__(self, gate = True, size_arg = "small", dropout = False, n_classes = 5):
        super(Model, self).__init__()
        self.size_dict = {"small": [2560, 512, 256], "big": [1024, 1, 384]}
        size = self.size_dict[size_arg]

        fc = [nn.Linear(size[0], size[1]), nn.ReLU()]
        if dropout:
            fc.append(nn.Dropout(0.25))

        if gate:
            attention_net = Attn_Net_Gated(L = size[1], D = size[2], dropout = dropout, n_classes = 1)

        else:
            attention_net = Attn_Net(L = size[1], D = size[2], dropout = dropout, n_classes = 1)
        
        fc.append(attention_net)
        self.attention_net = nn.Sequential(*fc)
        self.classifier = nn.Linear(size[1], n_classes)

        initialize_weights(self)

        self.criterion = nn.CrossEntropyLoss()
        #self.criterion = nn.NLLLoss()

        self.optimizer = torch.optim.Adam(self.parameters(),lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    def forward(self, h, return_features=False, attention_only=False):
        A, h = self.attention_net(h) 
        #print(A)
        A = torch.transpose(A, 1, 0) 
        
        if attention_only:
            return A
        A_raw = A 
        A = F.softmax(A, dim=1) 
        M = torch.mm(A, h)
       # print(A)
        logits  = self.classifier(M) 
        Y_hat = torch.topk(logits, 1, dim = 1)[1]
        Y_prob = F.softmax(logits, dim = 1)

        results_dict = {}
        if return_features:
            results_dict.update({'features': M})
        
        return logits, Y_prob, Y_hat, A_raw, results_dict

In [29]:
class MIL_Attention_fc_surv(Model):
    def __init__(self, gate = True, size_arg = "small", dropout = False, n_classes = 5):
        super(MIL_Attention_fc_surv, self).__init__(gate = gate, size_arg = size_arg, dropout = dropout, n_classes = n_classes)

    def forward(self, h, return_features=False, attention_only=False):
        A, h = self.attention_net(h)  
        A = torch.transpose(A, 1, 0) 
        if attention_only:
            return A
        A_raw = A 
        A = F.softmax(A, dim=1) 
        M = torch.mm(A, h) 
        logits  = self.classifier(M) 
        Y_hat = torch.topk(logits, 1, dim = 1)[1]
        hazards = torch.sigmoid(logits)
        S = torch.cumprod(1 - hazards, dim=1)
        results_dict = {}
        if return_features:
            results_dict.update({'features': M})
        
        return hazards, S, Y_hat, A_raw, results_dict

In [30]:
def train_one_epoch(model, data_loader, epoch):

    epoch_loss_train = 0.0
    n_batches = 0
    for data in tqdm(data_loader):
        model.optimizer.zero_grad()
        sequence_feature, labels, sequence_names = data

        sequence_feature = torch.squeeze(sequence_feature)
       # labels = labels.unsqueeze(0)
        
        if torch.cuda.is_available():
            features = Variable(sequence_feature.cuda())
            y_true = Variable(labels.cuda())
        else:
            features = Variable(sequence_feature)
            y_true = Variable(labels)
        #logits, Y_prob, Y_hat, _, results_dict = model(data)
        logits, y_pred, Y_hat, A, results_dict =  model(features)
        y_true = y_true
        y_pred = y_pred.mean(dim=0)
        Y_hat = torch.argmax(y_pred)
        #y_pred, dict_results = model(features)
        #Y_hat = Y_hat
        #print(logits)
        # calculate loss
        loss = model.criterion(logits, y_true.to(dtype=torch.long,non_blocking=False))
        #l2_lambda = 0.001
        #l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        #loss = loss + l2_lambda * l2_norm
        #print(loss)

        # backward gradient
        loss.backward()

        # update all parameters
        model.optimizer.step()

        epoch_loss_train += loss.item()
        n_batches += 1

    epoch_loss_train_avg = epoch_loss_train / n_batches
    #print(epoch_loss_train_avg)
    return epoch_loss_train_avg


In [31]:
def evaluate(model, data_loader):
    model.eval()

    epoch_loss = 0.0
    n_batches = 0
    valid_pred = []
    valid_true = []
    valid_name = []
    valid_score =[]

    for data in tqdm(data_loader):
        with torch.no_grad():
            sequence_feature, labels, sequence_names = data

            sequence_feature = torch.squeeze(sequence_feature)
            #labels = labels.unsqueeze(0)            
             
            if torch.cuda.is_available():
                features = Variable(sequence_feature.cuda())
               # labels = torch.squeeze(labels)

                y_true = Variable(labels.cuda())
            else:
                features = Variable(sequence_feature)
                y_true = Variable(labels)

            #y_pred, dict_results = model(features)
            logits, Y_prob, Y_hat, A, results_dict =  model(features)
            y_true = y_true
            Y_prob = Y_prob.mean(dim=0)
            Y_hat = torch.argmax(Y_prob)
        #y_pred, dict_results = model(features)
            #Y_hat = Y_hat
            #print(logits)

        # calculate loss
            #loss = model.criterion(Y_hat, y_true.long())
            #y_true = y_true

            loss = model.criterion(logits, y_true.to(dtype=torch.long,non_blocking=False))
            #l2_lambda = 0.001
            #l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
            #loss = loss + l2_lambda * l2_norm
            #print(loss)
                
            y_pred = Y_hat.cpu().detach().numpy().tolist()
            y_true = y_true.cpu().detach().numpy().tolist()
            valid_pred.append(y_pred)
            valid_true.append(y_true)
            valid_name.extend(sequence_names)
            valid_score.append(A)

            epoch_loss += loss.item()
            n_batches += 1
    epoch_loss_avg = epoch_loss / n_batches
    print(epoch_loss_avg)

    return epoch_loss_avg, valid_true, valid_pred, valid_name, valid_score


In [70]:
def train(model, dataframe,valid_dataframe,fold=0):
    train_loader = DataLoader(dataset=Dataset(train_dataframe) ,batch_size=1, shuffle=True, num_workers=0,collate_fn=collate_fn)
    valid_loader = DataLoader(dataset=Dataset(valid_dataframe) ,batch_size=1, shuffle=True, num_workers=0,collate_fn=collate_fn)

    train_losses = []
    train_binary_acc = []

    valid_losses = []
    valid_binary_acc = []

    best_val_loss = 1000
    best_epoch = 0

    for epoch in range(NUMBER_EPOCHS):
        print("\n========== Train epoch " + str(epoch + 1) + " ==========")
        model.train()

        epoch_loss_train_avg = train_one_epoch(model, train_loader, epoch + 1)
        print(epoch_loss_train_avg)
        #print("========== Evaluate Train set ==========")
        #_, train_true, train_pred, _, _ = evaluate(model, train_loader)
        #print(len(train_pred), len(train_pred)
        #result_train,binpt,bintt = analysis(train_true, train_pred)
        #print("Train binary acc: ", result_train['binary_acc'])

        #train_binary_acc.append(result_train['binary_acc'])
        print("========== Evaluate Valid set ==========")
        epoch_loss_valid_avg, valid_true, valid_pred, valid_name, valid_score = evaluate(model, valid_loader)
        result_valid, binp,bint = analysis(valid_true, valid_pred)
        print("Valid binary acc: ", result_valid['binary_acc'])
        valid_binary_acc.append(result_valid['binary_acc'])
        if best_val_loss > epoch_loss_valid_avg:
            best_val_loss = epoch_loss_valid_avg
            best_epoch = epoch + 1
            torch.save(model.state_dict(), os.path.join(Model_Path, 'Fold' + str(fold) + '_best_model.pkl'))
            valid_detail_dataframe = pd.DataFrame({'names': valid_name, 'stability': valid_true, 'prediction': valid_pred})
            valid_detail_dataframe.sort_values(by=['names'], inplace=True)
            valid_detail_dataframe.to_csv(Result_Path + 'Fold' + str(fold) + "_valid_detail.csv", header=True, sep=',')
 
    #result_all = {
     #   'Train_binary_acc': train_binary_acc,
     #   'Valid_binary_acc': valid_binary_acc,}
    #result = pd.DataFrame(result_all)
    #print("Fold", str(fold), "Best epoch at", str(best_epoch))
    #result.to_csv('result.csv')

def analysis(y_true, y_pred):
    #print(len(y_pred))
    binary_pred = y_pred
    #print(binary_pred, y_true)
    binary_true= y_true

   
    binary_acc = metrics.accuracy_score(binary_true, binary_pred)
    result = {

        'binary_acc': binary_acc,

    }
    return result , binary_pred, binary_true

def cross_validation(all_dataframe,fold_number=10):
    print("split_seed: ", SEED)
    sequence_names = all_dataframe['names'].values
    sequence_labels = all_dataframe['stability'].values
    kfold = KFold(n_splits=fold_number, shuffle=True)
    fold = 0

    for train_index, valid_index in kfold.split(sequence_names, sequence_labels):
        print("\n========== Fold " + str(fold + 1) + " ==========")
        train_dataframe = all_dataframe.iloc[train_index, :]
        valid_dataframe = all_dataframe.iloc[valid_index, :]
        print("Training on", str(train_dataframe.shape[0]), "examples, Validation on", str(valid_dataframe.shape[0]),
              "examples")
        model = Model()
        if torch.cuda.is_available():
            model.cuda()

        train(model, train_dataframe, valid_dataframe, fold + 1)
        fold += 1

In [71]:
cross_validation(ds,fold_number=5)

split_seed:  2333

========== Fold 1 ==========
Training on 575 examples, Validation on 144 examples

========== Train epoch 1 ==========


100%|████████████████████████████████████████████████████████████████████████████████| 719/719 [00:14<00:00, 48.76it/s]


1.5024624613627937
========== Evaluate Valid set ==========


100%|████████████████████████████████████████████████████████████████████████████████| 144/144 [00:01<00:00, 80.31it/s]


1.4972864815758333
Valid binary acc:  0.4027777777777778

========== Train epoch 2 ==========


...

0.5711946435072427
Valid binary acc:  0.8181818181818182

========== Train epoch 75 ==========


100%|████████████████████████████████████████████████████████████████████████████████| 719/719 [00:13<00:00, 52.09it/s]


0.6107135688594927
========== Evaluate Valid set ==========


100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [00:02<00:00, 67.03it/s]

0.555907157942542
Valid binary acc:  0.8251748251748252


In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [85]:
labels

array([2, 1, 1, 1, 2, 2, 4, 2, 3, 2, 3, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2, 3,
       3, 2, 0, 0, 4, 1, 3, 4, 2, 2, 0, 0, 1, 4, 4, 4, 2, 4, 1, 0, 3, 4,
       2, 1, 1, 4, 4, 4, 3, 1, 4, 1, 4, 4, 1, 2, 1, 2, 4, 1, 2, 1, 0, 0,
       4, 2, 1, 4, 4, 2, 4, 1, 1, 3, 1, 1, 4, 2, 4, 1, 1, 2, 1, 3, 2, 3,
       1, 1, 2, 2, 1, 4, 4, 0, 2, 1, 3, 1, 1, 2, 2, 0, 1, 1, 2, 2, 2, 4,
       2, 4, 3, 2, 1, 2, 1, 4, 1, 0, 1, 4, 1, 1, 4, 2, 1, 2, 1, 1, 4, 1,
       1, 1, 1, 1, 4, 3, 2, 2, 4, 4, 4, 1, 2, 3, 3, 2, 1, 1, 1, 1, 1, 4,
       4, 0, 2, 4, 1, 3, 2, 4, 4, 2, 4, 4, 0, 2, 3, 2, 2, 4, 1, 4, 4, 2,
       2, 2, 1, 1, 2, 1, 0, 0, 2, 1, 2, 1, 1, 2, 0, 2, 0, 4, 0, 1, 2, 2,
       2, 2, 2, 1, 1, 1, 3, 3, 4, 2, 0, 0, 3, 4, 1, 0, 4, 4, 1, 3, 1, 1,
       0, 1, 3, 3, 0, 1, 1, 3, 0, 0, 1, 1, 3, 4, 4, 4, 4, 2, 4, 4, 1, 2,
       4, 1, 2, 2, 1, 3, 1, 2, 4, 4, 3, 4, 4, 1, 4, 4, 3, 4, 2, 3, 1, 2,
       1, 0, 2, 4, 1, 1, 4, 1, 1, 1, 1, 1, 3, 3, 1, 1, 2, 2, 2, 4, 4, 1,
       2, 1, 2, 4, 2, 1, 1, 2, 4, 2, 1, 2, 3, 2, 2,

In [93]:
np.count_nonzero(labels == 4)

193

In [20]:
SEED = 2333
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.set_device(1)
    torch.cuda.manual_seed(SEED)

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [195]:
binary_pred

NameError: name 'binary_pred' is not defined

In [56]:
def get_simple_loader(dataset, batch_size=1):
	kwargs = {'num_workers': 0} if device.type == "cuda" else {}
	loader = DataLoader(dataset, batch_size=batch_size, sampler = sampler.SequentialSampler(dataset), collate_fn = collate_MIL, **kwargs)
	return loader 

In [21]:
import pickle
import torch
import numpy as np
import torch.nn as nn
import pdb

import torch
import numpy as np
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Sampler, WeightedRandomSampler, RandomSampler, SequentialSampler, sampler
import torch.optim as optim
import pdb
import torch.nn.functional as F
import math
from itertools import islice
import collections
device=torch.device("cpu")

class SubsetSequentialSampler(Sampler):
	"""Samples elements sequentially from a given list of indices, without replacement.
	Arguments:
		indices (sequence): a sequence of indices
	"""
	def __init__(self, indices):
		self.indices = indices

	def __iter__(self):
		return iter(self.indices)

	def __len__(self):
		return len(self.indices)

In [63]:
a = torch.Tensor([[1]])

In [69]:
a.squeeze()

tensor(1.)

In [72]:
for model_name in sorted(os.listdir(Model_Path)):
        print(model_name)
        model = Model()
        if torch.cuda.is_available():
            model.cuda()
        model.load_state_dict(torch.load(Model_Path + model_name,map_location='cpu'))

        #epoch_loss_test_avg, test_true, test_pred, test_name = evaluate(model, train_loader)

Fold1_best_model.pkl
Fold2_best_model.pkl
Fold3_best_model.pkl
Fold4_best_model.pkl
Fold5_best_model.pkl


In [75]:
def test(test_dataframe):
    test_loader = DataLoader(dataset=Dataset(ds) ,batch_size=1, shuffle=False, num_workers=0,collate_fn=collate_fn)
    test_result = {}

    for model_name in sorted(os.listdir(Model_Path)):
        print(model_name)
        model = Model()
        if torch.cuda.is_available():
            model.cuda()
        model.load_state_dict(torch.load(Model_Path + model_name,map_location='cpu'))

        epoch_loss_valid_avg, valid_true, valid_pred, valid_name, valid_score = evaluate(model, test_loader)
        #test_detail_dataframe = pd.DataFrame({'names': valid_name.cpu(), 'target': valid_true.cpu(), 'prediction': valid_pred.cpu(), 'Attention': valid_score.cpu()})
        
        return valid_score


In [76]:
a = test(ds)

Fold5_best_model.pkl


100%|████████████████████████████████████████████████████████████████████████████████| 719/719 [00:11<00:00, 64.73it/s]

0.5805949252531916


In [84]:
folders

['0096f71c-7777-417c-8d95-6fa1699879ec',
 '00be20a3-6ec8-44dd-afac-9085c6a03da9',
 '00ca6d3f-b495-4711-8ac3-d24175f5ee9f',
 '00f836a8-9b94-4413-95eb-fd127ee723b2',
 '01000de7-d0e8-4bb4-9918-4cbcd5bbfa3d',
 '0108fa63-6200-4896-8da9-0dbbd20998de',
 '011db747-84f4-4e3e-b446-1b667bef0ea9',
 '018aaaa8-78db-4341-8811-7356a871a2fe',
 '0222e12c-539f-4640-8007-fbd625fc0ea8',
 '024c0ec6-2e01-4640-a71c-6116e3a275ff',
 '029972c6-45c3-4a08-baf2-82a3b377fb71',
 '02b9de3a-c7ee-46ff-b0b7-e0561e6b3101',
 '0309db59-8d00-4c20-b816-015a3576f059',
 '039256d5-14c1-4057-b535-01416d2ba5ac',
 '0492d6eb-5192-45ca-949f-958e9f22c29d',
 '05042790-054b-43ea-a8bc-a8dd524c4627',
 '05792b42-0953-48ea-a4b1-3e9a1691677b',
 '0592845a-01d7-4d42-8149-72af28b83c53',
 '068e7a7f-7db9-4769-8052-33c03e1f7f32',
 '06a5cbd5-f9f4-466f-9f5a-a3fdea74f270',
 '07cee6c6-be65-47dd-b1d3-7a3311dec0a6',
 '07d54db2-b54b-4f87-a281-68f0de5328bb',
 '0834dfbb-1b2b-4125-a95b-d43a829c6fc7',
 '084893ca-b836-4824-a9b1-590980679155',
 '087647cd-4f1c-

In [49]:
len(a[100][0])

604

In [60]:
len(Atomic_wild[10])

2241

In [83]:
np.save('attnetion score.npy',k)

C:\Users\moham\anaconda3\envs\pytorch\lib\site-packages\numpy\lib\npyio.py:501: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [64]:
test_images = a[0].detach ().cpu ().numpy ()

In [85]:
np.save('name.npy',folders)

In [77]:
v=[]
for i in range(len(a)):
    v.append(a[i].detach ().cpu ().numpy ())

In [78]:
k = []
for i in range(len(a)):
    k.append(v[i][0])

In [89]:
score_dir = 'scores//'

for i in range(len(folders)):
    os.makedirs(score_dir + folders[i])
    with open(score_dir + folders[i] + '//score.pkl', 'wb') as f:
        pickle.dump(k[i], f) 

In [82]:
len(Atomic_wild[632])

11900